<span style="color:blue"><h1 style="text-align: center;" markdown="1">Lets chat with some humans!</h1></span>

![title](img/register.jpg)

Now it's time to get the human registered to visit a processing center.  This should be pretty easy, we just need to get them to select a location so CyberDyne-tech can allocation appropriate resources to that processing center.

Lets run the below code one cell at a time to build a chat bot in order to have a conversation with the human.


![title](img/Attention.png)
You <b>MUST</b> change the following variable values in the code cell below:
- slotName
- intentName
- botName

You can simply append your intials to the end of the variable value.

For instance:

change: 
```python
slotName = 'ProcessingLocations'
```
to:

```python
slotName = 'ProcessingLocationsXY'
```

(Bots, Intents and Slot names are unique to an account.  All lab participants are sharing a single account.  If you know you are using your own AWS credentials or credits, you do not need to rename the variables below)



In [ ]:
from boto3 import Session


session = Session(profile_name="default")
lex = session.client("lex-models")

# Change these variables to a unique value
botName = 'CyberDyneTech'
slotName = 'ProcessingLocations'
intentName = 'RegisterForCenterVisit'

# Don't change these variables.
slotDescription = 'Locations of processing centers'
intentDescription = 'Intent to register for a visit to a processing center'
intentUtterances = ['I would like to register to visit a center','I would like to visit a processing center', 'visit a center', 'visit', 'center']
botDescription = 'CyberDyne-Tech\'s registration chat bot'

In [ ]:
# Create the custom slot type
#try:
    # Delete the slot if it already exists
    #while delete_slot is None:
        #continue
    #print(delete_slot)
#except Exception:
    #pass

slot = lex.put_slot_type(name=slotName, description=slotDescription,
                             enumerationValues=[{'value': 'dallas'},{'value': 'austin'},{'value': 'san antonio'},{'value': 'houston'}])

print(slot)

In [ ]:
# Create an intent - using our Custom Slot type
intent = lex.put_intent(name=intentName, description=intentDescription,
                          slots=[
                              {
                                  'name': slotName,
                                  'description': slotDescription,
                                  'slotConstraint': 'Required',
                                  'slotType': slotName,
                                  'slotTypeVersion': '$LATEST',
                                  'valueElicitationPrompt': {
                                      'messages': [
                                          {
                                              'contentType': 'PlainText',
                                              'content': 'We have centers in all major Texas cities, which would you like to visit?'
                                          }
                                      ],
                                      'maxAttempts': 2
                                  },
                                  'priority': 1,
                                  'sampleUtterances':[]
                              }
                          ],
                          sampleUtterances=intentUtterances,
                          confirmationPrompt={
                              'messages': [
                                  {
                                      'contentType': 'PlainText',
                                      'content': 'So you are certain you wish to register for a visit at this center'
                                  },
                              ],
                              'maxAttempts': 2
                          },
                          rejectionStatement={
                            'messages': [
                                {
                                    'contentType': 'PlainText',
                                    'content': 'We are sorry,that facility is full at that time'
                                }
                            ]
                          },
                          conclusionStatement={
                              'messages': [
                                  {
                                      'contentType': 'PlainText',
                                      'content': 'Thank you for registering.  We can\'t wait to see you'
                                  },
                              ]
                          },
                          fulfillmentActivity={
                              'type': 'ReturnIntent'
                          },
                          parentIntentSignature='')

print(intent)


In [ ]:
# Create the ChatBot

bot = lex.put_bot(name=botName, description=botDescription,
                       intents=[
                           {
                            'intentName': intentName,
                            'intentVersion': '$LATEST'
                           },
                       ],
                       clarificationPrompt={
                           'messages':[
                               {
                                   'contentType': 'PlainText',
                                   'content': 'Sorry, I didn\'t understand that can you please try to say it a different way'
                               },
                           ],
                           'maxAttempts':3
                       },
                       abortStatement={
                           'messages':[
                               {
                                   'contentType': 'PlainText',
                                   'content': 'We are very sorry, but we are unable to understand your request and cannot continue. Good-Bye'
                               }
                           ]
                       },
                       idleSessionTTLInSeconds=60,
                       voiceId='Joanna',
                       processBehavior='BUILD',
                       locale='en-US',
                       childDirected=False)

print(bot)

Now that chatbot has been built - lets practice using it.

You will need to play the part of the Human, in addition to the bot

# Start the conversation with the chatbot by mentioning the phrase  "visit a center" - this is our <i> Intent </i>

In [ ]:
# Create a text box to enter our Chat text
botAlias = '$LATEST'
from ipywidgets import widgets
import random
lex = session.client("lex-runtime")

uID = str(random.randint(0,10000))
text = widgets.Text()
display(text)

def handle_submit(sender):
    response = lex.post_text(botName=botName, botAlias=botAlias, userId=uID, inputText=text.value)
    print(response['message'])
              
text.on_submit(handle_submit)


# In the code above we added Dallas, Austin, San Antonio and Houston as options.  Enter one of those cities here

In [ ]:
#enter a Texas city for the processing location

text_two = widgets.Text()
display(text_two)

def handle_submit(sender):
    response = lex.post_text(botName=botName, botAlias=botAlias, userId=uID, inputText=text_two.value)
    print(response['message'])
              
text_two.on_submit(handle_submit)

In [ ]:
text_three = widgets.Text()
display(text_three)

def handle_submit(sender):
    response = lex.post_text(botName=botName, botAlias=botAlias, userId=uID, inputText=text_three.value)
    print('You are registered!  We can\'t wait to see you at one of our processing centers!')
              
text_three.on_submit(handle_submit)

You're almost done with your mission!  Click [here](Finish.ipynb) for the final step.